<img src="../reports/figures/marketwatch.png" alt="Drawing" width="200">

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys   
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

<div class="alert alert-block alert-info">
    <b>Market Watch:</b>is a great websight to pull financial statements and economic reports. The goal is to use the information here
        as a 5 year trend to search for any stock on the webpage to pull in there well organized tables. 
    </div>

In [45]:
def get_financial_report(ticker):
    driver = webdriver.Firefox(executable_path= "geckodriver.exe")
    driver.maximize_window()
    driver.get('https://www.marketwatch.com/investing/stock/'+ticker+'/financials')
    timeout = 20
#     ''''
#     Find an ID on the page and wait before executing anything until found. After searching multiple times, 
#     You will be prompted to subscribe so there was a need to close out the subscribe overlay. 
#     ''''
    try:
        WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID, "cx-scrim-wrapper")))
    except TimeoutException:
        driver.quit()
    subscribe = driver.find_element_by_xpath('/html/body/footer/div[2]/div/div/div[1]')
    subscribe.click()
    
#     ''''
#     For the income statement, balance sheet, and cashflow;  you can scrape the tables and tranpose them to 
#     the years going 5 years back, for any calculations that might need to be done in the future. We kept all information
#     including empty columns just in case companies had more detailed reports and information. However for the 
#     most part all have the same format on market watch making it easily updateable and great for a search bar. 
#     ''''

    income_table = driver.find_element_by_class_name("overflow--table").get_attribute('innerHTML')
    income  = pd.read_html(income_table)
    income_df = income[0]
    income_df.drop(columns=['5-year trend'], axis=1, inplace=True)
    income_df2 = income_df.T
    income_new_col = income_df2.iloc[0,:].values
    income_df2.columns= income_new_col
    income_df2.drop(index='Item  Item', inplace=True)
    income_df2.rename(columns={'Sales/Revenue  Sales/Revenue': 'Sales/Revenue' , 
                               'Sales Growth  Sales Growth': 'Sales Growth',
           'Cost of Goods Sold (COGS) incl. D&A  Cost of Goods Sold (COGS) incl. D&A': 'Cost of Goods Sold (COGS) incl. D&A',
           'COGS Growth  COGS Growth': 'COGS Growth', 'COGS excluding D&A  COGS excluding D&A': 'COGS excluding D&A',
           'Depreciation & Amortization Expense  Depreciation & Amortization Expense': 'Depreciation & Amortization Expense',
           'Depreciation  Depreciation': 'Depreciation',
           'Amortization of Intangibles  Amortization of Intangibles': 'Amortization of Intangibles',
           'Gross Income  Gross Income': 'Gross Income',
           'Gross Income Growth  Gross Income Growth': 'Gross Income Growth',
           'Gross Profit Margin  Gross Profit Margin': 'Gross Profit Margin',
           'SG&A Expense  SG&A Expense': 'SG&A Expense', 'SGA Growth  SGA Growth': 'SGA Growth',
           'Research & Development  Research & Development': 'Research & Development',
           'Other SG&A  Other SG&A':  'Other SG&A',
           'Other Operating Expense  Other Operating Expense': 'Other Operating Expense',
           'Unusual Expense  Unusual Expense': 'Unusual Expense',
           'EBIT after Unusual Expense  EBIT after Unusual Expense': 'EBIT after Unusual Expense',
           'Non Operating Income/Expense  Non Operating Income/Expense': 'Non Operating Income/Expense',
           'Non-Operating Interest Income  Non-Operating Interest Income': 'Non-Operating Interest Income',
           'Equity in Affiliates (Pretax)  Equity in Affiliates (Pretax)': 'Equity in Affiliates (Pretax)',
           'Interest Expense  Interest Expense': 'Interest Expense',
           'Interest Expense Growth  Interest Expense Growth': 'Interest Expense Growth',
           'Gross Interest Expense  Gross Interest Expense': 'Gross Interest Expense',
           'Interest Capitalized  Interest Capitalized': 'Interest Capitalized',
           'Pretax Income  Pretax Income': 'Pretax Income',
           'Pretax Income Growth  Pretax Income Growth': 'Pretax Income Growth',
           'Pretax Margin  Pretax Margin': 'Pretax Margin', 'Income Tax  Income Tax': 'Income Tax',
           'Income Tax - Current Domestic  Income Tax - Current Domestic': 'Income Tax - Current Domestic',
           'Income Tax - Current Foreign  Income Tax - Current Foreign': 'Income Tax - Current Foreign',
           'Income Tax - Deferred Domestic  Income Tax - Deferred Domestic': 'Income Tax - Deferred Domestic',
           'Income Tax - Deferred Foreign  Income Tax - Deferred Foreign': 'Income Tax - Deferred Foreign',
           'Income Tax Credits  Income Tax Credits': 'Income Tax Credits',
           'Equity in Affiliates  Equity in Affiliates': 'Equity in Affiliates',
           'Other After Tax Income (Expense)  Other After Tax Income (Expense)': 'Other After Tax Income (Expense)',
           'Consolidated Net Income  Consolidated Net Income': 'Consolidated Net Income',
           'Minority Interest Expense  Minority Interest Expense': 'Minority Interest Expense',
           'Net Income  Net Income': 'Net Income', 'Net Income Growth  Net Income Growth': 'Net Income Growth',
           'Net Margin Growth  Net Margin Growth': 'Net Margin Growth',
           'Extraordinaries & Discontinued Operations  Extraordinaries & Discontinued Operations': 'Extraordinaries & Discontinued Operations',
           'Extra Items & Gain/Loss Sale Of Assets  Extra Items & Gain/Loss Sale Of Assets': 'Extra Items & Gain/Loss Sale Of Assets',
           'Cumulative Effect - Accounting Chg  Cumulative Effect - Accounting Chg': 'Cumulative Effect - Accounting Chg',
           'Discontinued Operations  Discontinued Operations': 'Discontinued Operations',
           'Net Income After Extraordinaries  Net Income After Extraordinaries': 'Net Income After Extraordinaries',
           'Preferred Dividends  Preferred Dividends': 'Preferred Dividends',
           'Net Income Available to Common  Net Income Available to Common': 'Net Income Available to Common',
           'EPS (Basic)  EPS (Basic)': 'EPS (Basic)', 'EPS (Basic) Growth  EPS (Basic) Growth': 'EPS (Basic) Growth',
           'Basic Shares Outstanding  Basic Shares Outstanding': 'Basic Shares Outstanding',
           'EPS (Diluted)  EPS (Diluted)': 'EPS (Diluted)',
           'EPS (Diluted) Growth  EPS (Diluted) Growth': 'EPS (Diluted) Growth',
           'Diluted Shares Outstanding  Diluted Shares Outstanding': 'Diluted Shares Outstanding',
           'EBITDA  EBITDA': 'EBITDA', 'EBITDA Growth  EBITDA Growth': 'EBITDA Growth',
           'EBITDA Margin  EBITDA Margin': 'EBITDA Margin'}, inplace=True)
    income_df2.to_csv('../data/interim/income_statement.csv')
    
    driver.get('https://www.marketwatch.com/investing/stock/'+ticker+'/financials/balance-sheet')
    balance_sheet_table = [table.get_attribute('innerHTML') for table in driver.find_elements_by_class_name("overflow--table")]
    balance_sheet = pd.read_html(balance_sheet_table[0])
    balance_sheet1 = pd.read_html(balance_sheet_table[1])
    balance_sheet_df = pd.concat([balance_sheet[0], balance_sheet1[0]])
    balance_sheet_df.drop(columns=['5-year trend'], axis=1, inplace=True)
    balance_sheet_df2 = balance_sheet_df.T
    balance_sheet_col = balance_sheet_df2.iloc[0,:].values
    balance_sheet_df2.columns= balance_sheet_col
    balance_sheet_df2.drop(index='Item  Item', inplace=True)
    balance_sheet_df2.rename(columns={'Cash & Short Term Investments  Cash & Short Term Investments': 'Cash & Short Term Investments',
           'Cash & Short Term Investments Growth  Cash & Short Term Investments Growth': 'Short Term Investments Growth',
           'Cash Only  Cash Only': 'Cash Only',
           'Short-Term Investments  Short-Term Investments': 'Short-Term Investments',
           'Cash & ST Investments / Total Assets  Cash & ST Investments / Total Assets': 'Cash & ST Investments / Total Assets',
           'Total Accounts Receivable  Total Accounts Receivable': 'Total Accounts Receivable',
           'Total Accounts Receivable Growth  Total Accounts Receivable Growth': 'Total Accounts Receivable Growth',
           'Accounts Receivables, Net  Accounts Receivables, Net': 'Accounts Receivables, Net',
           'Accounts Receivables, Gross  Accounts Receivables, Gross': 'Accounts Receivables, Gross',
           'Bad Debt/Doubtful Accounts  Bad Debt/Doubtful Accounts': 'Bad Debt/Doubtful Accounts',
           'Other Receivable  Other Receivable': 'Other Receivable',
           'Accounts Receivable Turnover  Accounts Receivable Turnover': 'Accounts Receivable Turnover',
           'Inventories  Inventories': 'Inventories', 'Finished Goods  Finished Goods': 'Finished Goods',
           'Work in Progress  Work in Progress': 'Work in Progress', 'Raw Materials  Raw Materials': 'Raw Materials',
           'Progress Payments & Other  Progress Payments & Other': 'Progress Payments & Other',
           'Other Current Assets  Other Current Assets': 'Other Current Assets',
           'Miscellaneous Current Assets  Miscellaneous Current Assets': 'Miscellaneous Current Assets',
           'Total Current Assets  Total Current Assets': 'Total Current Assets',
           'Net Property, Plant & Equipment  Net Property, Plant & Equipment': 'Net Property, Plant & Equipment',
           'Property, Plant & Equipment - Gross  Property, Plant & Equipment - Gross': 'Property, Plant & Equipment - Gross',
           'Buildings  Buildings': 'Buildings', 'Land & Improvements  Land & Improvements': 'Land & Improvements',
           'Computer Software and Equipment  Computer Software and Equipment': 'Computer Software and Equipment',
           'Other Property, Plant & Equipment  Other Property, Plant & Equipment': 'Other Property, Plant & Equipment',
           'Accumulated Depreciation  Accumulated Depreciation': 'Accumulated Depreciation',
           'Total Investments and Advances  Total Investments and Advances': 'Total Investments and Advances',
           'Other Long-Term Investments  Other Long-Term Investments': 'Other Long-Term Investments',
           'Long-Term Note Receivables  Long-Term Note Receivables': 'Long-Term Note Receivables',
           'Intangible Assets  Intangible Assets': 'Intangible Assets', 'Net Goodwill  Net Goodwill': 'Net Goodwill',
           'Net Other Intangibles  Net Other Intangibles': 'Net Other Intangibles',
           'Other Assets  Other Assets': 'Other Assets', 'Total Assets  Total Assets': 'Total Assets',
           'Total Assets Growth  Total Assets Growth': 'Total Assets Growth','ST Debt & Current Portion LT Debt  ST Debt & Current Portion LT Debt':'ST Debt & Current Portion LT Debt',
           'Short Term Debt  Short Term Debt': 'Short Term Debt',
           'Current Portion of Long Term Debt  Current Portion of Long Term Debt': 'Current Portion of Long Term Debt',
           'Accounts Payable  Accounts Payable': 'Accounts Payable',
           'Accounts Payable Growth  Accounts Payable Growth': 'Accounts Payable Growth',
           'Income Tax Payable  Income Tax Payable': 'Income Tax Payable',
           'Other Current Liabilities  Other Current Liabilities': 'Other Current Liabilities',
           'Dividends Payable  Dividends Payable': 'Dividends Payable',
           'Accrued Payroll  Accrued Payroll': 'Accrued Payroll',
           'Miscellaneous Current Liabilities  Miscellaneous Current Liabilities': 'Miscellaneous Current Liabilities',
           'Total Current Liabilities  Total Current Liabilities': 'Total Current Liabilities',
           'Long-Term Debt  Long-Term Debt': 'Long-Term Debt',
           'Long-Term Debt excl. Capitalized Leases  Long-Term Debt excl. Capitalized Leases': 'Long-Term Debt excl. Capitalized Leases',
           'Non-Convertible Debt  Non-Convertible Debt': 'Non-Convertible Debt',
           'Convertible Debt  Convertible Debt': 'Convertible Debt',
           'Capitalized Lease Obligations  Capitalized Lease Obligations': 'Capitalized Lease Obligations',
           'Provision for Risks & Charges  Provision for Risks & Charges': 'Provision for Risks & Charges',
           'Deferred Taxes  Deferred Taxes': 'Deferred Taxes',
           'Deferred Taxes - Credits  Deferred Taxes - Credits': 'Deferred Taxes - Credits',
           'Deferred Taxes - Debit  Deferred Taxes - Debit': 'Deferred Taxes - Debit',
           'Other Liabilities  Other Liabilities': 'Other Liabilities',
           'Other Liabilities (excl. Deferred Income)  Other Liabilities (excl. Deferred Income)': 'Other Liabilities (excl. Deferred Income)',
           'Deferred Income  Deferred Income': 'Deferred Income',
           'Total Liabilities  Total Liabilities': 'Total Liabilities',
           'Non-Equity Reserves  Non-Equity Reserves': 'Non-Equity Reserves',
           'Total Liabilities / Total Assets  Total Liabilities / Total Assets': 'Total Liabilities / Total Assets',
           'Preferred Stock (Carrying Value)  Preferred Stock (Carrying Value)': 'Preferred Stock (Carrying Value)',
           'Redeemable Preferred Stock  Redeemable Preferred Stock': 'Redeemable Preferred Stock',
           'Non-Redeemable Preferred Stock  Non-Redeemable Preferred Stock': 'Non-Redeemable Preferred Stock',
           'Common Equity (Total)  Common Equity (Total)': 'Common Equity (Total)',
           'Common Equity / Total Assets  Common Equity / Total Assets': 'Common Equity / Total Assets',
           'Common Stock Par/Carry Value  Common Stock Par/Carry Value': 'Common Stock Par/Carry Value',
           'Retained Earnings  Retained Earnings': 'Retained Earnings',
           'ESOP Debt Guarantee  ESOP Debt Guarantee': 'ESOP Debt Guarantee',
           'Cumulative Translation Adjustment/Unrealized For. Exch. Gain  Cumulative Translation Adjustment/Unrealized For. Exch. Gain': 'Cumulative Translation Adjustment/Unrealized For. Exch. Gain',
           'Unrealized Gain/Loss Marketable Securities  Unrealized Gain/Loss Marketable Securities': 'Unrealized Gain/Loss Marketable Securities',
           'Revaluation Reserves  Revaluation Reserves':  'Revaluation Reserves',
           'Treasury Stock  Treasury Stock': 'Treasury Stock',
           "Total Shareholders' Equity  Total Shareholders' Equity": 'Total Shareholders Equity',
           "Total Shareholders' Equity / Total Assets  Total Shareholders' Equity / Total Assets": 'Total Shareholders Equity / Total Assets',
           'Accumulated Minority Interest  Accumulated Minority Interest': 'Accumulated Minority Interest',
           'Total Equity  Total Equity': 'Total Equity',
           "Liabilities & Shareholders' Equity  Liabilities & Shareholders' Equity": 'Liabilities & Shareholders Equity'}, inplace=True)
    balance_sheet_df2.to_csv('../data/interim/balance_sheet.csv')
    
#     timeout = 20
#     # Find an ID on the page and wait before executing anything until found: 
#     try:
#         WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID, "cx-scrim-wrapper")))
#     except TimeoutException:
#         driver.quit()
#     subscribe = driver.find_element_by_xpath('/html/body/footer/div[2]/div/div/div[1]')
#     subscribe.click()

    driver.get('https://www.marketwatch.com/investing/stock/'+ticker+'/financials/cash-flow')
    cashflow_table = [table.get_attribute('innerHTML') for table in driver.find_elements_by_class_name("overflow--table")]
    cashflow = pd.read_html(cashflow_table[0])
    cashflow1 = pd.read_html(cashflow_table[1])
    cashflow_df = pd.concat([cashflow[0], cashflow1[0]]) 
    cashflow_df.drop(columns=['5-year trend'], axis=1, inplace=True)
    cashflow_df2 = cashflow_df.T
    cashflow_new_col = cashflow_df2.iloc[0,:].values
    cashflow_df2.columns= cashflow_new_col
    cashflow_df2.drop(index='Item  Item', inplace=True)
    cashflow_df2.rename(columns={'Net Income before Extraordinaries  Net Income before Extraordinaries': 'Net Income before Extraordinaries',
           'Net Income Growth  Net Income Growth': 'Net Income Growth',
           'Depreciation, Depletion & Amortization  Depreciation, Depletion & Amortization': 'Depletion & Amortization',
           'Depreciation and Depletion  Depreciation and Depletion': 'Depreciation and Depletion',
           'Amortization of Intangible Assets  Amortization of Intangible Assets': 'Amortization of Intangible Assets',
           'Deferred Taxes & Investment Tax Credit  Deferred Taxes & Investment Tax Credit': 'Deferred Taxes & Investment Tax Credit' ,
           'Deferred Taxes  Deferred Taxes': 'Deferred Taxes',
           'Investment Tax Credit  Investment Tax Credit': 'Investment Tax Credit',
           'Other Funds  Other Funds': 'Other Funds' ,
           'Funds from Operations  Funds from Operations': 'Funds from Operations',
           'Extraordinaries  Extraordinaries': 'Extraordinaries',
           'Changes in Working Capital  Changes in Working Capital': 'Changes in Working Capital',
           'Receivables  Receivables': 'Receivables', 'Accounts Payable  Accounts Payable': 'Accounts Payable',
           'Other Assets/Liabilities  Other Assets/Liabilities': 'Other Assets/Liabilities',
           'Net Operating Cash Flow  Net Operating Cash Flow': 'Net Operating Cash Flow',
           'Net Operating Cash Flow Growth  Net Operating Cash Flow Growth': 'Net Operating Cash Flow Growth',
           'Net Operating Cash Flow / Sales  Net Operating Cash Flow / Sales': 'Net Operating Cash Flow / Sales',
           'Capital Expenditures  Capital Expenditures': 'Capital Expenditures' ,
           'Capital Expenditures Growth  Capital Expenditures Growth': 'Capital Expenditures Growth',
           'Capital Expenditures / Sales  Capital Expenditures / Sales': 'Capital Expenditures / Sales',
           'Capital Expenditures (Fixed Assets)  Capital Expenditures (Fixed Assets)': 'Capital Expenditures (Fixed Assets)',
           'Capital Expenditures (Other Assets)  Capital Expenditures (Other Assets)': 'Capital Expenditures (Other Assets)',
           'Net Assets from Acquisitions  Net Assets from Acquisitions': 'Net Assets from Acquisitions',
           'Sale of Fixed Assets & Businesses  Sale of Fixed Assets & Businesses': 'Sale of Fixed Assets & Businesses',
           'Purchase/Sale of Investments  Purchase/Sale of Investments': 'Purchase/Sale of Investments',
           'Purchase of Investments  Purchase of Investments': 'Purchase of Investments',
           'Sale/Maturity of Investments  Sale/Maturity of Investments': 'Sale/Maturity of Investments',
           'Other Uses  Other Uses': 'Other Uses', 'Other Sources  Other Sources': 'Other Sources',
           'Net Investing Cash Flow  Net Investing Cash Flow': 'Net Investing Cash Flow',
           'Net Investing Cash Flow Growth  Net Investing Cash Flow Growth': 'Net Investing Cash Flow Growth',
           'Net Investing Cash Flow / Sales  Net Investing Cash Flow / Sales': 'Net Investing Cash Flow / Sales'}, inplace=True)
    cashflow_df2.to_csv('../data/interim/cashflows.csv')
    driver.quit()
    return income_df2, balance_sheet_df2, cashflow_df2

In [46]:
income_statement, balance_sheet, cashflow = get_financial_report('TSLA')

<div class="alert alert-block alert-info">
    <b>Income Statement</b>  is one of a company's core financial statements that shows their profit and lossProfit and Loss Statement (P&L)A profit and loss statement (P&L), or income statement or statement of operations, is a financial report that provides a summary of a over a period of time.<br><br>
    <b>Takeaway:</b>
        <dl>
            <dd><li>Net Income = (Total Revenue + Gains) – (Total Expenses + Losses)</li></dd>
            <dd><li>Total revenue is the sum of both operating and non-operating revenues while total expenses include those incurred by primary and secondary activities.</li></dd>
            <dd><li>Revenues are not receipts. Revenue is earned and reported on the income statement. Receipts (cash received or paid out) are not.</li></dd>
            <dd><li>An income statement provides valuable insights into a company’s operations, the efficiency of its management, under-performing sectors and its performance relative to industry peers.</li></dd>
         </dl>
</div>

<div class="alert alert-block alert-info">
    <b>Price-to-Earnings (P/E) Ratio</b>  Understanding earnings can be essential to determining the value of a company. To analyze earnings growth, investors can use earnings per share (EPS). This calculation measures the amount of earnings allocated to each share of stock. The P/E ratio is interpreted as a multiple, or measure, of how much an investor is paying for the stock compared to each dollar of a company’s annual earnings.
    <li><strong>Value investors</strong> are typically looking for stocks with a relatively low P/E ratio. This means they’re paying less for each dollar of earnings, which is a major characteristic of a value stock.</li>
    <dl>
        <dd>A cross-sectional analysis can provide more insight than the P/E ratio by itself. Comparing a company’s P/E ratio to a similar company or industry average can potentially reveal how well a company is actually performing. A good value stock candidate is likely to have a low P/E ratio relative to other companies in the same industry. </dd><br>
        <dd>P/E ratio referred to as a trailing P/E ratio or a forward P/E ratio. The difference is in the source of the earnings figure. A trailing P/E uses the company’s last 12 months of earnings, while a forward P/E uses an estimate of the company’s next 12 months of earnings.</dd><br>
        <dd>If a company uses aggressive accounting, the P/E ratio may appear artificially low. That’s why it can be important to also use other ratios like the price-to-sales and price-to-book ratios</dd>
    </dl>
    <li><strong>Growth investors</strong> are typically looking for stocks with a relatively high P/E ratio.</li>
    <img src="../reports/figures/pe_ratio.png" alt="Drawing" width="600"></img>
</div>

<div class="alert alert-block alert-info">
    <b>Price-to-Sales (P/S) Ratio</b>  The price-to-sales (P/S) ratio compares a stock’s price to, you guessed it, the company’s annual sales per share. This is a simple measure of how much revenue the company brings in from its sales activity. <br>
    <li><strong>Value investors</strong> are typically looking for stocks with a relatively low P/S ratio. The P/S ratio is interpreted as a multiple of how much an investor is paying for the stock compared to each dollar of a company’s annual sales.</li>
    <dl>
        <dd>One downside to using a P/S ratio is that sales numbers can often vary greatly from year-to-year or even quarter-to-quarter. Furthermore, it may seem redundant to compare price to such similar metrics as sales (P/S ratio) and earnings (P/E ratio).</dd>
    </dl>
    <img src="../reports/figures/ps_ratio.png" alt="Drawing" width="400"></img>
</div>

<div class="alert alert-block alert-info">
    <b>Price-to-Book (P/B) Ratio</b>  The P/B ratio compares the stock price to a company’s book value, which is the per-share value of shareholders’ equity.<strong> Shareholders’ equity, or book value,</strong> is a balance sheet item equal to the company’s assets minus liabilities.<br>
    <b>The book value growth rate</b>, or the percentage increase in shareholders’ equity over time, can reveal important clues about a company’s strength: an increasing book value might indicate an increase in shareholders’ equity. You typically want to see a company’s book value grow over time.<br>
    <img src="../reports/figures/bv.png" alt="Drawing" width="400"></img>
    <li><strong>Value investors</strong> generally prefer to invest in stocks with a low P/B ratio. This can indicate that they’re paying a lower multiple for the equity in the business.</li>
    <dl>
        <dd>Book value may not accurately represent the economic value of a company’s assets for two main reasons. First, book value ignores the economic value of intangible assets like brands or intellectual property. Second, book values may not reflect various accounting practices and how companies choose to record assets on their balance sheets. Despite its limitations, the P/B ratio can be a useful tool in evaluating and searching for value stocks when combined with other ratios.</dd>
    </dl>
    <img src="../reports/figures/pb_ratio.png" alt="Drawing" width="400"></img>
</div>


In [47]:
income_statement

,Sales/Revenue,Sales Growth,Cost of Goods Sold (COGS) incl. D&A,COGS Growth,COGS excluding D&A,Depreciation & Amortization Expense,Depreciation,Amortization of Intangibles,Gross Income,Gross Income Growth,...,Net Income Available to Common,EPS (Basic),EPS (Basic) Growth,Basic Shares Outstanding,EPS (Diluted),EPS (Diluted) Growth,Diluted Shares Outstanding,EBITDA,EBITDA Growth,EBITDA Margin
2015,4.05B,-,3.12B,-,2.7B,422.59M,-,-,923.5M,-,...,(888.66M),(1.39),-,641.01M,(1.39),-,641.01M,(294.04M),-,-
2016,7B,73.01%,5.45B,74.40%,4.5B,947.1M,-,-,1.55B,68.31%,...,(674.91M),(0.94),32.47%,721.06M,(0.94),32.47%,721.06M,256.56M,187.25%,-
2017,11.76B,67.98%,9.54B,75.22%,7.91B,1.64B,-,-,2.22B,42.62%,...,(1.96B),(2.37),-152.78%,828.79M,(2.37),-152.78%,828.79M,(1.68M),-100.66%,-
2018,21.46B,82.51%,17.42B,82.56%,15.52B,1.9B,-,-,4.04B,82.33%,...,(976.09M),(1.14),51.61%,852.63M,(1.14),51.61%,852.63M,1.65B,"98,032.86%",-
2019,24.58B,14.52%,20.51B,17.74%,18.36B,2.15B,-,-,4.07B,0.67%,...,(862M),(0.97),14.92%,885M,(0.97),14.92%,885M,2.23B,35.54%,9.09%


({'EPS (Basic)': 'eps','EPS (Basic) Growth': 'epsgrowth','Net Income': 'netincome','Total Shareholders\' Equity': 'shareholderequity','roa': roa,'longtermdebt': longtermdebt,'interestexpense': interestexpense,'ebitda': ebitda},index=range(date.today().year-5,date.today().year))

<div class="alert alert-block alert-info">
    <b>Balance Sheet</b> is a financial statement that reports a company's assets, liabilities and shareholders' equity. The balance sheet is one of the three (income statement and statement of cash flows being the other two) core financial statements used to evaluate a business.<br><br>
        <b>Takeaway:</b>
            <dl>
                <dd><li>The balance sheet is one of the three (income statement and statement of cash flows being the other two) core financial statements used to evaluate a business.</li></dd>
                <dd><li>The balance sheet is a snapshot, representing the state of a company's finances (what it owns and owes) as of the date of publication.</li></dd>
                <dd><li>Fundamental analysts use balance sheets, in conjunction with other financial statements, to calculate financial ratios.</li></dd>
           </dl>
</div>

In [48]:
balance_sheet

,Cash & Short Term Investments,Short Term Investments Growth,Cash Only,Short-Term Investments,Cash & ST Investments / Total Assets,Total Accounts Receivable,Total Accounts Receivable Growth,"Accounts Receivables, Net","Accounts Receivables, Gross",Bad Debt/Doubtful Accounts,...,ESOP Debt Guarantee,Cumulative Translation Adjustment/Unrealized For. Exch. Gain,Unrealized Gain/Loss Marketable Securities,Revaluation Reserves,Treasury Stock,Total Shareholders Equity,Total Shareholders Equity / Total Assets,Accumulated Minority Interest,Total Equity,Liabilities & Shareholders Equity
2015,1.22B,-,1.22B,-,15.12%,168.97M,-,168.97M,168.97M,-,...,-,-,-,-,-,1.08B,13.43%,-,1.08B,8.07B
2016,3.5B,186.89%,3.5B,-,15.44%,499.14M,195.41%,499.14M,499.14M,-,...,-,-,-,-,-,4.75B,20.97%,1.15B,5.91B,22.66B
2017,3.52B,0.70%,3.52B,-,12.30%,515.38M,3.25%,515.38M,515.38M,-,...,-,-,-,-,-,4.24B,14.79%,1.4B,5.63B,28.66B
2018,3.88B,10.07%,3.88B,-,13.04%,949.02M,84.14%,949.02M,949.02M,-,...,-,-,-,-,-,4.92B,16.55%,1.39B,6.31B,29.74B
2019,6.51B,67.97%,6.51B,-,18.99%,1.32B,39.51%,1.32B,1.32B,-,...,-,-,-,-,-,6.62B,19.29%,1.49B,8.11B,34.31B


<div class="alert alert-block alert-info">
    <b>Cash Flow Statement</b> a cash flow statement, also known as statement of cash flows, is a financial statement that shows how changes in balance sheet accounts and income affect cash and cash equivalents, and breaks the analysis down to operating, investing, and financing activities.<br><br>
            <b>Takeaway:</b>
            <dl>
                <dd><li>The cash flow statement includes cash made by the business through operations, investment, and financing—the sum of which is called net cash flow.</li></dd>
                <dd><li>The first section of the cash flow statement is cash flow from operations, which includes transactions from all operational business activities.</li></dd>
                <dd><li>Cash flow from investment is the second section of the cash flow statement, and is the result of investment gains and losses.</li></dd>
                 <dd><li>Cash flow from financing is the final section, which provides an overview of cash used from debt and equity.</li></dd>
           </dl>
</div>

In [49]:
cashflow

,Net Income before Extraordinaries,Net Income Growth,Depletion & Amortization,Depreciation and Depletion,Amortization of Intangible Assets,Deferred Taxes & Investment Tax Credit,Deferred Taxes,Investment Tax Credit,Other Funds,Funds from Operations,...,Net Assets from Acquisitions,Sale of Fixed Assets & Businesses,Purchase/Sale of Investments,Purchase of Investments,Sale/Maturity of Investments,Other Uses,Other Sources,Net Investing Cash Flow,Net Investing Cash Flow Growth,Net Investing Cash Flow / Sales
2015,(888.66M),-,422.59M,-,-,-,-,-,434.86M,(31.21M),...,(12.26M),-,(26.44M),(26.44M),-,-,-,(1.67B),-,-41.36%
2016,(773.05M),13.01%,947.1M,-,-,-,-,-,395.98M,570.03M,...,-,415.05M,(189.48M),(206.15M),16.67M,-,-,(1.21B),27.41%,-17.36%
2017,(2.24B),-189.84%,1.64B,-,-,-,-,-,1.04B,435.95M,...,(114.52M),789.7M,(223.09M),(223.09M),-,-,287.21M,(3.63B),-198.73%,-30.86%
2018,(1.06B),52.58%,1.9B,-,-,-,-,-,1.2B,2.04B,...,(17.91M),437.13M,-,-,-,-,-,(1.9B),47.64%,-8.85%
2019,(775M),27.06%,2.15B,-,-,-,-,-,1.38B,2.75B,...,(45M),279M,-,-,-,-,-,(1.16B),39.11%,-4.71%


<div class="alert alert-block alert-success">
    <b>income_statement, balance_sheet, cashflow = get_financial_report('VLO')</b> is a function that 
    generates these reports through a ticker input. A 5 year report to search for any stock on the webpage to pull in there well organized tables. 
    </div>

# Fundamental Analysis

<div class="alert alert-block alert-info">
    <b>Fundamental Analysis:</b> is an approach to identifying investment opportunities. Fundamental investors use financial statements, economic reports, and forecasts to examine the key drivers of a company’s current and future potential business activities.<br>
    <br>
    <b>Fundamental Investors:</b> Fundamental investors often take a holistic approach by examining all of the company’s <strong>strengths, weaknesses, opportunities, and threats</strong>. These items form the basis of an investment thesis, and uncover areas that may require additional analysis.<br>
    <br>
    <li><strong>Strengths</strong> are areas that the business does well, like a company with high net profit margins.</li>
    <li><strong>Weaknesses</strong> are areas that the business doesn’t do well, like carrying too much debt.</li>
    <li><strong>Opportunities</strong> are areas that can likely be improved rather quickly.</li>
     <li><strong>Threats</strong> are areas where competitors are potentially able to outperform a company, like with a newly developed
         product.</li><br>
    <br>
     <b>Long-Term Business Trends:</b> less influenced by short-term noise. For example, imagine a company in an up-and-coming industry that has secured a 10-year patent on a revolutionary technology.<br>
    <br>
    <b>Assumptions:</b> estimating a stock’s intrinsic value or future growth potential requires making assumptions. But a stock’s market value may never catch up with the value you determine.<br>
     <br>
    <b>Forecasting Errors:</b> An error can come from many sources, including inaccurate data, accounting blunders, or simply mathematical mistakes.
    </div>

<div class="alert alert-block alert-danger">
    <b>get_financial_reports(ticker):</b> The original function was meant to just use the following libraries<strong> requests, bs4, and datetime,</strong> however, it became apparent if any alteration was done to the site, it didn't seem proficent compared to using something like <strong>Selenium.</strong>
</div>

In [ ]:
import requests
from datetime import *
from bs4 import BeautifulSoup as bs

In [ ]:
def get_financial_reports(ticker):
#build URLs
    urlfinancials = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials'
    urlbalancesheet = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials/balance-sheet'
#request the data using beautiful soup
    text_soup_financials = BeautifulSoup(requests.get(urlfinancials).text,"html") 
    text_soup_balancesheet = BeautifulSoup(requests.get(urlbalancesheet).text,"html")
    # build lists for Income statement
    titlesfinancials = text_soup_financials.findAll('td', {'class': 'rowTitle'})
    epslist=[]
    netincomelist = []
    longtermdebtlist = [] 
    interestexpenselist = []
    ebitdalist= []
    #load data into lists if the row title is found
    for title in titlesfinancials:
        if 'EPS (Basic)' in title.text:
            epslist.append ([td.text for td in  title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Net Income' in title.text:
            netincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Interest Expense' in title.text:
            interestexpenselist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'EBITDA' in title.text:
            ebitdalist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
    # find the table headers for the Balance sheet
    titlesbalancesheet = text_soup_balancesheet.findAll('td', {'class': 'rowTitle'})
    equitylist=[]
    for title in titlesbalancesheet:
        if 'Total Shareholders\' Equity' in title.text:
            equitylist.append( [td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Long-Term Debt' in title.text:
            longtermdebtlist.append( [td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
    #get the data from the income statement lists 
    #use helper function get_element
    eps = get_element(epslist,0)
    epsGrowth = get_element(epslist,1)
    netIncome = get_element(netincomelist,0)
    shareholderEquity = get_element(equitylist,0)
    roa = get_element(equitylist,1)
    longtermDebt = get_element(longtermdebtlist,0)
    interestExpense =  get_element(interestexpenselist,0)
    ebitda = get_element(ebitdalist,0)

    # load all the data into dataframe 
    fin_df= pd.DataFrame({'eps': eps,'eps Growth': epsGrowth,'net Income': netIncome,'shareholder Equity': shareholderEquity,'roa': roa,'longterm Debt': longtermDebt,'interest Expense': interestExpense,'ebitda': ebitda}, index=range(date.today().year-5,date.today().year))

    fin_df.reset_index(inplace=True)

    return fin_df

In [ ]:
#helper function
def get_element(list,element):
    try:
        return list[element]
    except:
        return '-'         